### Code Pangea - Pondera Lab: data generation by entity (Aguascalientes, México)



* Install Python libraryes 

In [ ]:
!pip install geopandas

In [ ]:
!pip install jenkspy

In [ ]:
!pip install feature_engine

In [ ]:
!pip install fitter

In [ ]:
!pip install imblearn

In [ ]:
!pip install yellowbrick==0.9.1 scikit-learn==0.24 #22.2

### Stage 1

Segmentation of households from the ENIGH 2018 CONCENTRADOHOGAR base, the most important variables are selected and their natural cuts are obtained, thus building the *Segmentations of PANGEA*, this first part we create the `segmentation.csv` base that contains the information of the natural breaks (with the Jenks algorithm) of the important variables. This forms the segmentation or Tapestry, this base is modified in the second part to obtain a more ad-hoc segmentation.

In [9]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import time

import geopandas as gpd

import joblib
import pickle
from joblib import dump, load

import warnings
warnings.simplefilter("ignore")

In [10]:
# Jenks
import jenkspy
from gvf import classify, goodness_of_variance_fit, percentage_class_members

# Funciones para obtener las bases de datos
from enigh_db2 import get_enigh
from census2 import fill_zeros,get_cvegeo,prepare_census

# Funiones para el pre proceso de los datos
from pre_proceso import FrecuentImputer, MeanImputer

### Data reading and exploratory analysis

Read the ENIGH and Census database for the appropriate state.

* The *get_enigh* function created in the *enigh_db* document, obtains the ENIGH base with the necessary features by joining the additional bases.
     The additional bases are poverty, IDH and municipalities (base created by us with information from the municipality).
     * The entire base can be ordered or just for one state.
     * Filter only for towns with >=2500 inhabitants.
     * The output is the base with the unions and the list of variables that are used for prediction.
     * These variables are those that are intercepted with the census.
* The function *prepare_census* of the document *census.py*, returns the census ready to be applied.

In [ ]:
estado = 1 # state number (Aguascalientes, México)

pobreza = True 
idh = True
municipios = True

# Enigh
enigh, FEATURES = get_enigh(estado=estado, pobreza=pobreza, idh=idh, municipios=municipios)
enigh.shape

List of variables separated by their origin, If not all of them are used, the list of FEATURES can be changed.

In [12]:
municipios = ['huelgas_est', 'longitud_mun', 'calentador_sol', 'med_privad', 'separar_res', 'insi_trafico', 'por_jubilados', 'quehaceres_hog', 'pca3_mun',
              'conflictor_tr', 'latitud_mun', 'pca2_mun', 'pca1_mun', 'focos_ahorr']

idh = ['anios_esp_esc', 'indice_ingreso', 'indice_edu', 'mortalidad_inf', 'indice_salud', 'anios_prom_esc', 'ingreso_anual_capita', 'valor_idh']

pobreza = ['ingreso_inferior_lineabienmin', 'no_vul_no_pob', 'ingreso_inferior_lineabien', 'tres_carencia_social', 'carencia_basicos_viv',
           'carencia_calidad_vivienda', 'carencia_acceso_alimento', 'una_carencia_social', 'coeficiente_gini', 'pobre_extrema', 'carencia_acceso_salud',
           'pobre_moderada', 'carencia_acceso_segsocial', 'pobre', 'carencia_rezago_edu', 'vulnerable_social', 'vulnerable_ingreso']

coneval = ['ingreso_inferior_lineabien', 'tres_carencia_social', 'carencia_basicos_viv', 'indice_ingreso', 'indice_salud', 'pobre', 'ingreso_inferior_lineabienmin',
           'anios_esp_esc', 'mortalidad_inf', 'anios_prom_esc', 'vulnerable_social', 'no_vul_no_pob', 'carencia_calidad_vivienda', 'una_carencia_social',
           'coeficiente_gini', 'pobre_moderada', 'ingreso_anual_capita', 'valor_idh', 'vulnerable_ingreso', 'carencia_acceso_alimento', 'pobre_extrema',
           'carencia_acceso_salud', 'indice_edu', 'carencia_acceso_segsocial', 'carencia_rezago_edu']

#FEATURES = list(set(FEATURES) - set(pobreza+idh))

Loading the census base for the corresponding state

In [ ]:
censo = prepare_census(estado=estado)
censo.shape

In [ ]:
# Checking for duplicate values
enigh[['folioviv', 'foliohog']].drop_duplicates().shape, enigh.shape, censo['CVEGEO'].unique().shape, censo.shape

### Clustering segmentation tapestry

Zeros in **current income**, which is the first variable to rank
* The number of zeros in the variable is printed.
* If there is at least one zero in the current input, the first cantile is added, so as not to have problems when doing the Box-Cox transformation

In [ ]:
n_zeros = (enigh['ing_cor']==0).sum()
print('No. ceros: ',n_zeros)
if n_zeros>0:
    q_min = np.quantile(enigh['ing_cor'],0.01)
    print('Cuantil: ', q_min)
    enigh.loc[enigh['ing_cor']==0,'ing_cor'] = q_min

*Apply the Box Cox transformation to normalize the data.*

In [16]:
xt = enigh['ing_cor'].values
#x = enigh['ing_cor'].values
"""pt = PowerTransformer(method='box-cox',standardize=False)
pt.fit(x.reshape(-1,1))
xt = pt.transform(x.reshape(-1,1)).reshape(-1)"""

enigh['ing_cor_t'] = xt

**Optional:** Caping to remove outlayers
* The values of the 0.01 quantile in current income are replaced by the previous value.

In [ ]:
"""from feature_engine.outliers import Winsorizer

capper = Winsorizer(capping_method='quantiles',fold=0.01,tail='both',variables='ing_cor')

enigh = capper.fit_transform(enigh)"""

### Jenks Natural Breaks 

The metric used to evaluate how good the segmentation is is *Goodness of variance fit (GVF)*

*You can change the gvf (how well the variability is distributed).*

#### ENIGH variable 

We use the variables **ing_cor** (current income), the variables **tot_integ** (household members), and **employed** (number of members who work) to carry out the Tapestry, are segmented at the national level.

In [ ]:
gvf = 0.0
nclasses = 1
while gvf < .97:
    nclasses += 1
    breaks = jenkspy.jenks_breaks(xt, nclasses)
    breaks = np.array(breaks)
    gvf = goodness_of_variance_fit(xt, breaks)
    ppc = percentage_class_members(xt,breaks)
    print('No. de clases: {} | GVF: {}'.format(nclasses,round(gvf,2)))
    print('Cortes naturales: ',[np.round(xx,2) for xx in breaks])
    #print('Cortes naturales escala original: ',[np.round(xx/60,2) for xx in pt.inverse_transform(breaks.reshape(-1,1)).reshape(-1)])
    print('Porcentaje de miembros por clase: {}\n'.format(ppc))

By selecting the number of cuts, 6 cuts will be used, but you can:
* Using more cuts and joining some segments
* Using the 6 cuts given by Jenks

In [17]:
nclasses = 8
breaks_ing_cor = jenkspy.jenks_breaks(xt, nclasses)
breaks_ing_cor = np.array(breaks_ing_cor)

Current income histogram with selected cutoffs

In [ ]:
plt.figure(figsize=[15,5])
plt.hist(xt)
for v in breaks_ing_cor:
    plt.axvline(v,color='black')
plt.show()

Percentage of elements in each cut.

In [ ]:
def find_segment(value,intervals_list): 
    n = len(intervals_list)
    segment = np.nan
    for j in range(1,n-1):
        if value<=intervals_list[j]:
            segment = j-1
            break
        elif value<=intervals_list[j+1]:
            segment = j
            break
    if value<intervals_list[0]:
        segment = np.nan
    return segment

enigh['ing_cor_t_segment'] = enigh['ing_cor_t'].apply(find_segment,args=(breaks_ing_cor,))

(enigh['ing_cor_t_segment'].value_counts(normalize=True)*100).sort_index()

segment reassignment

In [ ]:
enigh['ing_cor_t_segment'] = enigh['ing_cor_t_segment'].replace({6:5, 7:5, 8:5})#, 9:5

(enigh['ing_cor_t_segment'].value_counts(normalize=True)*100).sort_index()

Fixing the segments to save them

In [ ]:
# copying the first values
breaks_ing_cor_new = breaks_ing_cor[0:7].copy()
# copying the first values
breaks_ing_cor_new[0] = 0
# The last segment ends in the largest value, actually any larger value
breaks_ing_cor_new[-1] = breaks_ing_cor[-1] + 100000

for i in range(len(breaks_ing_cor_new)-1):
  print('Segmento: ', i, ' --- Intervalo: [{}, {}]'.format(breaks_ing_cor_new[i], breaks_ing_cor_new[i+1]))


** save archives

In [ ]:
breaks_dic = {'breaks_ingreso':breaks_ing_cor_new}

nombre = '/content/gdrive/MyDrive/pamgea3_0/pangea3_2/breacks/ingreso_estado'+str(estado)+'.json'
joblib.dump(breaks_dic, nombre)

# joblib.load(nombre)

### Stage 2

* Applying the classification supervised learning model to the variable **ing_cor_t_segment**
* The variables that are found both in the census bases and in the enigh are used, adding information on Idh, Poverty, municipalities (Denue).

Checking if there is null data in any variable

In [ ]:
enigh.isnull().sum().sort_values(ascending=False)

In [ ]:
censo.isnull().sum().sort_values(ascending=False)

Preprocessing the variables:
* Removing null values in some census variables
* For the INT_FEATURES variables, the large values in the census base are delimited and replaced by the highest value found in the enigh.\
     (This is done because the census uses a synthetic household that is the average, so the average should not be greater than the maximum.)
* The variable is printed, the average of the variables above the maximum in the enigh and the amount of data that exceeds that limit.


In [ ]:
# If the value in the census is greater than the maximum of the enigh, it is bounded.
INT_FEATURES = [
    'tot_integ',
    'ocupados',
    'menores',
    'p65mas',
    'hombres',
    'mujeres',
    'mayores',
    'p12_64'
    ]

censo = censo.dropna(subset=INT_FEATURES).reset_index(drop=True)

for c in INT_FEATURES:
        censo[c] = censo[c].apply(lambda x: np.ceil(x) if np.modf(x)[0]>=0.5 else np.floor(x))
        censo[c] = censo[c].replace({np.inf:0})
        censo[c] = censo[c].astype(int)

for c in INT_FEATURES: 
    mmax = enigh[c].max() 
    e_stat = enigh[c].max()
    e_stat = np.ceil(e_stat) if np.modf(e_stat)[0]>=0.5 else np.floor(e_stat) # integer
    # modf: Return the fractional and integral parts of an array
    print(c,': ',np.round((censo[c]>mmax).mean()*100,4),(censo[c]>mmax).sum())
    censo.loc[censo[c]>mmax,c] = e_stat

Creating a new variable

In [ ]:
enigh['tot_ocu'] = enigh['ocupados']/enigh['tot_integ']
censo['tot_ocu'] = censo['ocupados']/censo['tot_integ']

In [ ]:
FEATURES = FEATURES+['tot_ocu']

### Adding pre-processing so there is no null data in the census

In [ ]:
# categorical variables
categ = ['sexo_jefe', 'vph_telef', 'vph_cel', 'vph_inter', 'vph_stvp', 'vph_autom', 'vph_moto', 'vph_bici', 'vph_radio',
        'vph_tv', 'vph_lavad', 'vph_refri', 'vph_hmicro', 'vph_pc', 'vph_cvj', 'vph_ndacmm', 'vph_snbien', 'vph_sinrtv',
        'vph_sinltc', 'vph_sincint', 'pea', 'pder_segp', 'pder_ss', 'pder_imss', 'pder_istee','vph_pisodt', 'vph_pisoti', 'vph_1dor',
        'vph_2ymasd', 'vph_1cuart', 'vph_2cuart', 'vph_3ymasc', 'vph_aguadv', 'vph_excsa', 'vph_tinaco', 'vph_cister', 
        'vph_drenaj', 'vph_c_elec','huelgas_est']   # list(set() - set(pobreza+idh))

# Poverty and HDI variables
pob_idh =['pobre', 'pobre_moderada', 'pobre_extrema', 'vulnerable_social', 'vulnerable_ingreso', 'no_vul_no_pob',
          'una_carencia_social', 'tres_carencia_social', 'carencia_rezago_edu', 'carencia_acceso_salud',
          'carencia_acceso_segsocial', 'carencia_calidad_vivienda', 'carencia_basicos_viv', 'carencia_acceso_alimento',
          'ingreso_inferior_lineabien', 'ingreso_inferior_lineabienmin', 'coeficiente_gini', 'anios_esp_esc',
          'ingreso_anual_capita', 'mortalidad_inf', 'indice_edu', 'indice_salud', 'indice_ingreso', 'valor_idh']

# Those that necessarily have to be applied log
log_seg = ['por_jubilados']

# log 
log_pos = ['mayores', 'p15ym_an', 'pca1_mun', 'pca3_mun', 'pca2_mun', 'mujeres', 'anios_prom_esc', 'ocupados']

# Jeo o Box Cox 
inv = ['quehaceres_hog', 'focos_ahorr', 'med_privad', 'separar_res', 'pca1_mun', 'p12_64', 'tot_integ', 'insi_trafico', 'hombres',
        'conflictor_tr', 'pob15_64']

continuas = list(set(FEATURES)-set(categ))

Null values

In [ ]:
# Studying the number of missing data by categorical variable

enigh = enigh.replace([np.inf, -np.inf], np.nan)

cat_vars_with_na = [
    var for var in categ
    if enigh[var].isnull().sum() > 0
]

# print percentage of missing values per variable
enigh[cat_vars_with_na ].isnull().mean().sort_values(ascending=False)

In [ ]:
censo[list(set(FEATURES)-set(categ))].isnull().sum().sum()

In [ ]:
# Passing infinite values to null
enigh = enigh.replace([np.inf, -np.inf], np.nan)
#enigh[categ] = enigh[categ].fillna("Missing")
#enigh[categ] = enigh[categ].astype('O')
    
censo = censo.replace([np.inf, -np.inf], np.nan)
#censo[categ] = censo[categ].fillna("Missing")
#censo[categ] = censo[categ].astype('O')

Separating training and testing

In [ ]:
from sklearn.model_selection import train_test_split

X = enigh[FEATURES]
y = enigh[['ing_cor_t_segment']]

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

X_train.shape, X_test.shape

Training the classification algorithm to predict *ing_cor_t_segment*, on the variables found above, using the ENIGH base.

* Pre-processing to variables
     * Treat null data
     * Encoding the categorical data (not always necessary)
     * Transformation to variables
     * Scale the data (in case it is necessary for the model)
* Balance the data if the model does not
* Apply a classification model (good results have been obtained with RF, but you can change the model)
* Optimize hyper parameters:
     * Under F1 or Recall macro

Note: For best results, for example, {0:5, 1:5, 2:5, 3:4, 4:3, 5:4}

In [ ]:
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline 

# feature engenearing
from feature_engine.selection import DropConstantFeatures, DropDuplicateFeatures, DropCorrelatedFeatures
#from sklearn.impute import SimpleImputer,IterativeImputer
from feature_engine.encoding import OrdinalEncoder, RareLabelEncoder, CountFrequencyEncoder
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from feature_engine import transformation as vt
from sklearn.preprocessing import StandardScaler

# feature selectio 
from sklearn.feature_selection import RFE

# Model
#from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from imblearn.over_sampling import RandomOverSampler 
from sklearn.ensemble import RandomForestClassifier
#from imblearn.ensemble import BalancedRandomForestClassifier

# Metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

# Procesing
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from feature_engine.encoding import CountFrequencyEncoder


start = time.time()

model1 = RandomForestClassifier(random_state=42)
#model2 = BalancedRandomForestClassifier(random_state=42)


params = {
    # Feature selection
    'feature_sel__n_features_to_select':np.linspace(10, 40, 20).astype(int).tolist(),
    # Model
    'classifier__n_estimators':np.linspace(start = 50, stop = 2000, num = 20).astype(int).tolist(), 
    'classifier__max_depth':[None]+np.linspace(5, 110, num = 15).astype(int).tolist(), 
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf':[1, 2, 4, 10],
    'classifier__bootstrap': [True, False],
    'classifier__max_features':['auto','sqrt'],
    'classifier__criterion':['gini','entropy']
}


pipe = Pipeline([
    # Mising imputation
    ('categorical_imputer', FrecuentImputer(variables = categ)),
    ('median_imputer', MeanImputer(variables = continuas)),
    # Fearute selection
    ('feature_sel', RFE(estimator = model1)),
    # Model
    ('oversample', RandomOverSampler(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42, class_weight={0:5, 1:4, 2:5, 3:5, 4:20, 5:10})), #
])
clf = RandomizedSearchCV(pipe,
                         params,
                         n_iter=5, # Iteration number
                         n_jobs=-1,
                         scoring='f1_weighted',#'recall_macro'
                         random_state=42)


clf.fit(X_train.astype(float), y_train.values.reshape(-1))
print(clf.best_score_)

end = time.time()
print('Running time: {:.2f} minutes'.format((end-start)/60))

In [ ]:
clf.best_params_

In [ ]:
selector = RFE(estimator = model1, n_features_to_select=38)
selector.fit(X_train, y_train)
X_train.columns[selector.support_]

In [ ]:
print('idh:', set(idh).intersection(set(X_train.columns[selector.support_])))
print('\n Pobreza:', set(pobreza).intersection(set(X_train.columns[selector.support_])))
print('\n Municipios:', set(municipios).intersection(set(X_train.columns[selector.support_])))

Metrics and confusion matrices, pay more attention to the figures.

* The differences between the confusion matrices is the way they are normalized:
     * All: Among the total elements (Acuracy)
     * Predict: (Presition)
     * True: Each row / total row (Recall)


In [ ]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

print('TRAIN')
cr = classification_report(y_train,y_train_pred)
print(cr)

print('TEST')
cr = classification_report(y_test,y_test_pred)
print(cr)


cm = confusion_matrix(y_test,y_test_pred,normalize='true')
plt.figure()
plt.title('NORMALIZED = TRUE')
plt.imshow(cm,cmap='cividis')
plt.show()

cm = confusion_matrix(y_test,y_test_pred,normalize='pred')
plt.figure()
plt.title('NORMALIZED = PREDICT')
plt.imshow(cm,cmap='cividis')
plt.show()

cm = confusion_matrix(y_test,y_test_pred,normalize='all')
plt.figure()
plt.title('NORMALIZED = ALL')
plt.imshow(cm,cmap='cividis')
plt.show()

Comparing the number of elements in the test base for each of the classes, training vs real.

In [ ]:
real_segment_vc = y_test['ing_cor_t_segment'].value_counts()
pred_segment_vc = pd.Series(y_test_pred).value_counts()

real_segment_vc = pd.DataFrame(real_segment_vc).sort_index().reset_index(drop=True)
real_segment_vc.columns = ['real_count']

pred_segment_vc = pd.DataFrame(pred_segment_vc)
pred_segment_vc.columns = ['pred_count']

real_segment_vc.merge(pred_segment_vc,left_index=True,right_index=True).sort_index().plot.bar(figsize=[10,5])

Applying the created metric

In [ ]:
met = []
for i in range(6):
    met.append((real_segment_vc.loc[i].values-pred_segment_vc.loc[i].values)/(real_segment_vc.loc[i].values+pred_segment_vc.loc[i].values))
    print('Segmento ', i,':', met[i])

val=0
for i in range(6):
    val = val + np.abs(met[i])
print('Métrica general:', val/6)

val=0
for i in range(6):
    val = val + np.abs(met[i])*(real_segment_vc.loc[i].values/real_segment_vc.sum().values)
print('Métrica balanceada:', val)

Predicting the Census CVGEO class
* The model is retrained with all the ENIGH data
* Training and VC metrics are returned
* The census is classified. Printing the frequency of each of its segments

Save as a Pickle

In [ ]:
clf_best = clf.best_estimator_
best_score = clf.best_score_
print('Best score: {}'.format(best_score))

filename = '/content/gdrive/MyDrive/pamgea3_0/pangea3_2/modelos/estado'+str(estado)+'.pkl'    ### 'estado'+str(estado)+'.pkl', for a single municipalityeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
pickle.dump(clf_best, open(filename, 'wb'))

Predicting values for the Census

In [ ]:
censo['ing_cor_t_segment'] = clf.predict(censo[FEATURES].reset_index(drop=True))
censo['ing_cor_t_segment'].value_counts(normalize=True)*100

Graph of the importance of the variables

In [ ]:
try:
    imp = pd.DataFrame({'imp':clf.best_estimator_.feature_importances_,'features':X.columns.tolist()})
    imp = imp.set_index('features')
    imp = imp.sort_values(by='imp')
    imp.plot.barh()
except:
    try:
        imp = pd.DataFrame({'imp':clf.best_estimator_.steps[-1][1].feature_importances_,'features':X.columns.tolist()})
        imp = imp.set_index('features')
        imp = imp.sort_values(by='imp')
        imp.plot.barh()
    except:
        pass

Comparing the proportion of data in enigh vs census.

* There is usually more data in class three. Since the average household is too average (with income = 3) and because we are giving it more weight

In [ ]:
censo_pred = censo['ing_cor_t_segment'].value_counts(normalize=True)*100
enigh_true = enigh['ing_cor_t_segment'].value_counts(normalize=True)*100
censo_pred = pd.DataFrame(censo_pred)
enigh_true = pd.DataFrame(enigh_true)
censo_pred.columns = ['censo']
enigh_true.columns = ['enigh']
enigh_true.merge(censo_pred,left_index=True,right_index=True).sort_index().plot.bar(figsize=[10,5])

In [ ]:
met = []
for i in range(6):
    met.append((enigh_true.loc[i].values-censo_pred.loc[i].values)/(enigh_true.loc[i].values+censo_pred.loc[i].values))
    print('Segmento ', i,':', met[i])

val=0
for i in range(6):
    val = val + np.abs(met[i])
print('Métrica general:', val/6)

val=0
for i in range(6):
    val = val + np.abs(met[i])*(real_segment_vc.loc[i].values/real_segment_vc.sum().values)
print('Métrica balanceada:', val)

Graphing the municipalities to buy with google maps. Compare in big cities.

In [ ]:
from matplotlib.collections import PatchCollection

estado_ = '0'+str(estado) if estado<10 else str(estado)
shp = gpd.read_file(f'/content/gdrive/MyDrive/pamgea3_0/shape_estados/{estado_}m.shp')
shp.to_crs('EPSG:4326',inplace=True)

# list of municipalities
muns = ['001']

if len(muns)>0:
    shp = shp[shp['CVE_MUN'].isin(muns)].reset_index(drop=True)
    filename = '/content/gdrive/MyDrive/pamgea3_0/pangea3_2/mapas/ing_cor_estado'+str(estado)+'_municipio'+str(muns)+'.png'
else:
    filename = '/content/gdrive/MyDrive/pamgea3_0/pangea3_2/mapas/ing_cor_estado'+str(estado)+'.png'
    
shp = shp.merge(censo,left_on='CVEGEO',right_on='CVEGEO')

# shp = shp[shp['ing_cor_t_segment'].isin([2,3,4,5])]
base = shp.plot(column='ing_cor_t_segment',figsize=[15,15],legend=True)

plt.savefig(filename)

**Saving probabilities for each census class**

In [ ]:
a = clf.predict_proba(censo[FEATURES].loc[:, ~censo[FEATURES].columns.duplicated()])

for i in range(6):
    censo['clase_'+str(i)] = a[:, i]

In [ ]:
df2 = censo[['CVEGEO', 'tot_integ', 'ocupados', 'ing_cor_t_segment','clase_0', 'clase_1', 'clase_2', 'clase_3', 'clase_4', 'clase_5']].copy()
df2.shape

In [ ]:
df2.head()

In [ ]:
df2.to_csv('/content/gdrive/MyDrive/pamgea3_0/pangea3_2/fase2/proba_ing_cor_'+str(estado)+'.csv', index=False)

Saving the training and test bases, in case you want to study the test metrics for the income and expense variables.

In [ ]:
# Select the training and test data
entrenamiento = enigh.loc[X_train.index].copy()
prueba = enigh.loc[X_test.index].copy()

print(entrenamiento.shape, prueba.shape)

# Calculating the probabilities for the training and test bases
a_train = clf.predict_proba(X_train[FEATURES])
a_test = clf.predict_proba(X_test[FEATURES])

for i in range(6):
    entrenamiento['clase_'+str(i)] = a_train[:, i]
    prueba['clase_'+str(i)] = a_test[:, i]

entrenamiento['ing_cor_t_segment'] = entrenamiento[['clase_0', 'clase_1', 'clase_2', 'clase_3', 'clase_4', 'clase_5']].idxmax(axis="columns").replace({'clase_0':0, 'clase_1':1, 'clase_2':2, 'clase_3':3, 'clase_4':4, 'clase_5':5})
prueba['ing_cor_t_segment'] = prueba[['clase_0', 'clase_1', 'clase_2', 'clase_3', 'clase_4', 'clase_5']].idxmax(axis="columns").replace({'clase_0':0, 'clase_1':1, 'clase_2':2, 'clase_3':3, 'clase_4':4, 'clase_5':5})

print(entrenamiento.shape, prueba.shape)

entrenamiento.to_csv('/content/gdrive/MyDrive/pamgea3_0/pangea3_2/fase2/entrenamiento'+str(estado)+'.csv', index=False)
prueba.to_csv('/content/gdrive/MyDrive/pamgea3_0/pangea3_2/fase2/prueba'+str(estado)+'.csv', index=False)

### Stage 3:
At this stage the tapestry for the state will be generated.

* Initially it defines the tapestry on the enigh and a grouping is done by its values.
* The tapestry is defined for each block in the census.
* Finally, the value of bock expenditure for the census is obtained.

(This step can be automated for all states)

### 3.1 Tapestry in Enigh Reading the file with the definition of the cuts made in stage 1. In case you don't have the enigh grouping, you must do this step.

In [71]:
# Census data
censo = pd.read_csv('/content/gdrive/MyDrive/pamgea3_0/pangea3_2/fase2/proba_ing_cor_'+str(estado)+'.csv')

In [72]:
# Current income cuts at the state level
nombre = '/content/gdrive/MyDrive/pamgea3_0/pangea3_2/breacks/ingreso_estado'+str(estado)+'.json'
breacks = joblib.load(nombre)
breaks_ingreso = breacks['breaks_ingreso']

# Cuts in total number of people and employed persons per household at the national level
breacks = joblib.load('/content/gdrive/MyDrive/pamgea3_0/breaks1/breaks_dic_nacional.json')
breaks_tot_integ = breacks['breaks_tot_integ']
breaks_ocupados = breacks['breaks_ocupados']

In [ ]:
breaks_ingreso, breaks_tot_integ, breaks_ocupados

In [ ]:

def find_segment(value,intervals_list): 
    n = len(intervals_list)
    segment = np.nan
    for j in range(1,n-1):
        if value<=intervals_list[j]:
            segment = j-1
            break
        elif value<=intervals_list[j+1]:
            segment = j
            break
    if value<intervals_list[0]:
        segment = np.nan
    return segment

breaks2 = [breaks_tot_integ, breaks_ocupados, breaks_ingreso]
variables2 = ['tot_integ', 'ocupados', 'ing_cor']

for i in range(3):
    enigh[variables2[i]+'_segment'] = enigh[variables2[i]].apply(find_segment,args=(breaks2[i],))
    print(variables2[i])
    print((enigh[variables2[i]+'_segment'].value_counts(normalize=True)*100).sort_index())

# Si son nan es por que están por ensima del valor máximo
enigh.loc[enigh['ing_cor_segment'].isnull(), 'ing_cor_segment'] = 5
enigh['ing_cor_segment'] = enigh['ing_cor_segment'].astype(int)

In [75]:
# Creating el tapestry
enigh['tapestry'] = enigh['ing_cor_segment'].astype(str)+'_'+enigh['tot_integ_segment'].astype(str)+'_'+enigh['ocupados_segment'].astype(str)

Deciding if we delete some groups

In [ ]:
print(enigh.loc[enigh['tapestry'].str[0] == '4', 'tapestry'].shape)
enigh.loc[enigh['tapestry'].str[0] == '4', 'tapestry'].value_counts()

In [ ]:
print(enigh.loc[enigh['tapestry'].str[0] == '5', 'tapestry'].shape)
enigh.loc[enigh['tapestry'].str[0] == '5', 'tapestry'].value_counts()

**Passing the elements of the last two classes to `4_*_*` and `5_*_*`**

**Note:** The graph is very different from the previous one, since the last two groups, even when they were united, had almost no segments

In [78]:
enigh.loc[enigh['tapestry'].str[0] == '4', 'tapestry'] = '4_*_*'
enigh.loc[enigh['tapestry'].str[0] == '5', 'tapestry'] = '5_*_*'

In [ ]:
# Compare tapestry
real_segment_vc = enigh['tapestry'].value_counts(normalize=True)*100

real_segment_vc = pd.DataFrame(real_segment_vc).sort_index()

real_segment_vc.sort_index().plot.bar(figsize=[20, 6])
plt.title('Tapestry del enigh')
plt.show()

### 3.2.1 Distribution of the number of people per household

Instead of putting 100% in that the total number of members is in the mean:
* The state (or municipal) distribution of the variable is obtained, from the enigh
* For each block, the distribution obtained from enihg is created, with the average in the corresponding census value, and the probability that households are in each of the segments is calculated.
* These probabilities are multiplied with the other probabilities of the tapestry, so as to obtain the probability that the typical household of each block belongs to each of the variables.

In [ ]:
val = enigh['tot_integ'].value_counts()

ing = enigh['tot_integ'].tolist()

for i in range(len(val.index)):
    ing= ing + list(np.repeat(np.linspace(val.index[i]+0.15, val.index[i]+0.85, 6), val.values[i]))
    
plt.hist(ing, bins=50)

In [ ]:
from scipy import stats
from fitter import Fitter

start = time.time()

f = Fitter(ing)#enigh['tot_integ'].values)
f.fit()

end = time.time()
print('Running time: {:.2f} minutes'.format((end-start)/60))

f.summary()

In [ ]:
# best approximation
#f.get_best(method='sumsquare_error')
f.fitted_param['mielke']

In [ ]:
from pylab import linspace, plot
import scipy.stats

dist = scipy.stats.mielke
param = (1.2141543114975002, 5.649409918554957, 0.9684464558884465, 4.929259920874483)
X = linspace(0,14, 50)
pdf_fitted = dist.cdf(X, *param)
plot(X, pdf_fitted, 'o-')

Calculating the probability of the number of people per household in each block

In [87]:
breaks2 = [breaks_tot_integ, breaks_ocupados]
variables2 = ['tot_integ', 'ocupados']

In [ ]:
# how much is added to the average

anteriores = ['tot_integ_clase0', 'tot_integ_clase1', 'tot_integ_clase2']

for i in list(censo['tot_integ'].value_counts().index):
    mean = i - 0.9684464558884465 - 1 #1.7
    for j in range(4):
        if j==0:
            val = dist.cdf(breaks_tot_integ[j+1], k=1.2141543114975002, s=5.649409918554957, loc=mean, scale= 4.929259920874483)
            censo.loc[censo['tot_integ']==i, 'tot_integ_clase'+str(j)] = val
        elif j==3:
            val = 1 - dist.cdf(breaks_tot_integ[j], k=1.2141543114975002, s=5.649409918554957, loc=mean, scale= 4.929259920874483)
            censo.loc[censo['tot_integ']==i, 'tot_integ_clase'+str(j)] = val #[x if x >0 else 0 for x in val]
        else:
            val = dist.cdf(breaks_tot_integ[j+1], k=1.2141543114975002, s=5.649409918554957, loc=mean, scale= 4.929259920874483) - dist.cdf(breaks_tot_integ[j], k=1.2141543114975002, s=5.649409918554957, loc=mean, scale= 4.929259920874483)
            censo.loc[censo['tot_integ']==i, 'tot_integ_clase'+str(j)] = val

censo['tot_integ_clase'] = censo[['tot_integ_clase0', 'tot_integ_clase1', 'tot_integ_clase2', 'tot_integ_clase3']].idxmax(axis="columns").replace({'tot_integ_clase0':0, 'tot_integ_clase1':1, 'tot_integ_clase2':2, 'tot_integ_clase3':3})

censo['tot_integ_clase'] = censo[['tot_integ_clase0', 'tot_integ_clase1', 'tot_integ_clase2', 'tot_integ_clase3']].idxmax(axis="columns").replace({'tot_integ_clase0':0, 'tot_integ_clase1':1, 'tot_integ_clase2':2, 'tot_integ_clase3':3})

# Graphic
x = np.array(list(range(4)))
p = (censo['tot_integ_clase'].value_counts(normalize=True)*100).sort_index().values
e = (enigh['tot_integ_segment'].value_counts(normalize=True)*100).sort_index().values
  
plt.bar(x - 0.15, p, 0.3, label = 'Censo')
plt.bar(x + 0.15, e, 0.3, label = 'Enigh')
plt.xticks(x, x)
plt.legend()
plt.show()


In [ ]:
# Getting a random number that follows the distribution
for i in list(censo['tot_integ'].value_counts().index):
  mean = i - 0.9684464558884465 - 1 #1.7
  censo.loc[censo['tot_integ']==i, 'tot_integ_sim'] = dist.rvs(k=1.2141543114975002, s=5.649409918554957, loc=mean, scale= 4.929259920874483, size=sum(censo['tot_integ']==i), random_state=1234)

# Adjusting to be integers greater than zero
censo['tot_integ_sim'] = round(censo['tot_integ_sim'], 0).astype(int)
censo['tot_integ_sim'] = np.where(censo['tot_integ_sim']<0, 1, censo['tot_integ_sim'])

# Getting the ranking
censo['tot_integ_segment_sim'] = censo['tot_integ_sim'].apply(find_segment,args=(breaks_tot_integ,))

censo['tot_integ_segment_sim'] = np.where(censo['tot_integ_sim']==0, 0, censo['tot_integ_segment_sim'])
censo['tot_integ_segment_sim'] = np.where(censo['tot_integ_sim']>=breaks_tot_integ[-1], 3, censo['tot_integ_segment_sim'])

# Observing the results
x = np.array(list(range(4)))
p = (censo['tot_integ_segment_sim'].value_counts(normalize=True)*100).sort_index().values
e = (enigh['tot_integ_segment'].value_counts(normalize=True)*100).sort_index().values
  
plt.bar(x - 0.15, p, 0.3, label = 'Censo')
plt.bar(x + 0.15, e, 0.3, label = 'Enigh')
plt.xticks(x, x)
plt.legend()
plt.show()

### 3.2.1 Distribution for the number of people and employed per household


In [ ]:
val = enigh['ocupados'].value_counts()

ocu = enigh['ocupados'].tolist()

for i in range(len(val.index)):
    ocu= ocu + list(np.repeat(np.linspace(val.index[i]+0.15, val.index[i]+0.85, 5), val.values[i]))
    
plt.hist(ocu, bins=50)

In [ ]:
from scipy import stats
from fitter import Fitter

start = time.time()

f = Fitter(ocu)#enigh['tot_integ'].values)
f.fit()

end = time.time()
print('Running time: {:.2f} minutes'.format((end-start)/60))

f.summary()

In [ ]:
# La mejor aproximación
#f.get_best(method='sumsquare_error')
f.fitted_param['skewnorm']

In [ ]:
#b = f.get_best(method='sumsquare_error').values()
#list(list(b)[0].values())

In [ ]:
from pylab import linspace, plot
import scipy.stats

dist = scipy.stats.skewnorm
param = (2.9191685095369353, 0.8006910433468928, 1.815458588743899)
X = linspace(0,14, 50)
pdf_fitted = dist.cdf(X, *param)
plot(X, pdf_fitted, 'o-')

In [ ]:
# Set how much to add to the mean

anteriores = ['ocupados_clase0', 'ocupados_clase1', 'ocupados_clase2']

for i in list(censo['ocupados'].value_counts().index):
    mean = i - 0.8006910433468928 - 0.5

    for j in range(4):
        if j==0:
            val = dist.cdf(breaks_tot_integ[j+1], a=5.10782986052344, loc=mean, scale=1.815458588743899)
            censo.loc[censo['ocupados']==i, 'ocupados_clase'+str(j)] = val
        elif j==3:
            val = 1 - dist.cdf(breaks_tot_integ[j], a=5.10782986052344, loc=mean, scale=1.815458588743899)
            censo.loc[censo['ocupados']==i, 'ocupados_clase'+str(j)] = val #[x if x >0 else 0 for x in val]
        else:
            val = dist.cdf(breaks_tot_integ[j+1], a=5.10782986052344, loc=mean, scale=1.815458588743899) - dist.cdf(breaks_ocupados[j], a=5.10782986052344, loc=mean, scale=1.815458588743899)
            censo.loc[censo['ocupados']==i, 'ocupados_clase'+str(j)] = val

censo['ocupados_clase'] = censo[['ocupados_clase0', 'ocupados_clase1', 'ocupados_clase2', 'ocupados_clase3']].idxmax(axis="columns").replace({'ocupados_clase0':0, 'ocupados_clase1':1, 'ocupados_clase2':2, 'ocupados_clase3':3})

plt.hist([censo['ocupados_clase'], enigh['ocupados_segment']], label=['Censo', 'Enigh'], density=True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Getting a random number that follows the distribution
for i in list(censo['ocupados'].value_counts().index):
  mean = i - 0.8006910433468928 - 0.5
  censo.loc[censo['ocupados']==i, 'ocupados_sim'] = dist.rvs(a=5.10782986052344, loc=mean, scale=1.815458588743899, size=sum(censo['ocupados']==i), random_state=1234)

# Adjusting to be integers greater than zero
censo['ocupados_sim'] = round(censo['ocupados_sim'], 0).astype(int)
censo['ocupados_sim'] = np.where(censo['ocupados_sim']<0, 1, censo['ocupados_sim'])

# Getting the classification
censo['ocupados_segment_sim'] = censo['ocupados_sim'].apply(find_segment,args=(breaks_ocupados,))

censo['ocupados_segment_sim'] = np.where(censo['ocupados_sim']==0, 0, censo['ocupados_segment_sim'])
censo['ocupados_segment_sim'] = np.where(censo['ocupados_sim']>=breaks_ocupados[-1], 3, censo['ocupados_segment_sim'])

# Getting the classification
plt.hist([censo['ocupados_segment_sim'], enigh['ocupados_segment']], label=['Censo', 'Enigh'], density=True)
plt.legend(loc='upper right')
plt.show()  

### 3.2.2 Tapestry for synthetic probability of home and employed

In [ ]:
# Delimiting the number of people employed by the number of people in the home
censo['ocupados_sim'] = np.where(censo['ocupados_sim'] > censo['tot_integ_sim'] , censo['tot_integ_sim'], censo['ocupados_sim']).astype(int)

# Segmenting the busy variable
censo['ocupados_segment_sim'] = censo['ocupados_sim'].apply(find_segment,args=(breaks_ocupados,))

# Observing the results
plt.hist([censo['ocupados_segment_sim'], enigh['ocupados_segment']], label=['Censo', 'Enigh'], density=True)
plt.legend(loc='upper right')
plt.show()  

In [94]:
# If there is a null value, then they belong to the last segment
censo.loc[censo['ocupados_segment_sim'].isnull(), 'ocupados_segment_sim'] = 3
censo['ocupados_segment_sim'] = censo['ocupados_segment_sim'].astype(int)

In [ ]:
# Creating tapestry
censo['ing_cor_t_segment'] = censo['ing_cor_t_segment'].astype(int)
censo['tapestry'] = censo['ing_cor_t_segment'].astype(str)+'_'+censo['tot_integ_segment_sim'].astype(int).astype(str)+'_'+censo['ocupados_segment_sim'].astype(str)
print(censo['tapestry'].unique().shape, censo.shape)

censo.loc[censo['tapestry'].str[0] == '4', 'tapestry'] = '4_*_*'
censo.loc[censo['tapestry'].str[0] == '5', 'tapestry'] = '5_*_*'
print(censo['tapestry'].unique().shape)

In [ ]:
# Creating a list of all possible segments (excluding _*_*)

from itertools import product

x = [str(i)+'_' for i in range(4)]
y = [str(i)+'_' for i in range(4)]
lista1 = [i+j for i, j in product(x, y)]

z = [str(i) for i in range(4)]
lista = [i+j for i, j in product(lista1, z)]

#lista
censo_tapestry = pd.DataFrame(columns=['CVEGEO']+lista+['4_*_*', '5_*_*'])
censo_tapestry['CVEGEO'] = censo['CVEGEO'].copy()
censo_tapestry['tapestry'] = censo['tapestry'].copy()
censo_tapestry['est'] = censo['CVEGEO'].str[:2].copy()
censo_tapestry['mun'] = censo['CVEGEO'].str[2:5].copy()

censo_tapestry.shape

In [97]:
censo['tot_integ_segment_sim'] = censo['tot_integ_segment_sim'].astype(int)

In [ ]:
clases = ['clase_0',	'clase_1',	'clase_2',	'clase_3',	'clase_4',	'clase_5']

# Creating tapestry
for i in range(4): # number of participants
  for j in range(4): # busy
    for k in range(6): # income segment
      cond = (censo['tot_integ_segment_sim'] == i) & (censo['ocupados_segment_sim'] == j) #& (prueba['ing_cor_t_segment'] == k)
      if k<=3:
        col = str(k)+'_'+str(i)+'_'+str(j)
        censo_tapestry.loc[cond, col] = censo.loc[cond, 'clase_'+str(k)].copy()
      else:
        censo_tapestry.loc[cond, str(k)+'_*_*'] = censo.loc[cond, 'clase_'+str(k)].copy()
      print(col)

# Null values are set equal to zero
censo_tapestry = censo_tapestry.fillna(0)

# Eliminating inconsistent cases between members and employees
for i in range(4):
    censo_tapestry[str(i)+'_0_2'] = censo_tapestry[str(i)+'_0_2'] + censo_tapestry[str(i)+'_0_3'].fillna(0)
    censo_tapestry = censo_tapestry.drop(columns=[str(i)+'_0_3'])

In [ ]:
# Checking that the probability of the tapestry is one
a = censo_tapestry.drop(columns=['CVEGEO', 'tapestry', 'est', 'mun']).sum(axis=1)
sum((a<1.0001) & (a>0.9999)), len(a)

In [ ]:
# Graphing the results
acumulado = censo['tapestry'].value_counts()

enigh_ac = pd.DataFrame(acumulado)

enigh_ac.sort_index().plot.bar(figsize=[20,5])

### 3.3 Fixing the enigh variables, so that all segments of the enigh tapestry are had in the censo

In [ ]:
# Studying if there is any segment of the tapestry without probabilities
col_no_vacias_entrenamiento = list(enigh['tapestry'].unique())
print('Número de elementos distintos del tapestry en entrenamiento:', len(col_no_vacias_entrenamiento))  

col_no_vacias_censo = []
for i in censo_tapestry.drop(columns=['CVEGEO', 'tapestry', 'est', 'mun']).columns:
    if censo_tapestry[i].sum() != 0:
        col_no_vacias_censo.append(i)
        
print('Número de elementos distintos del tapestry en el censo:', len(col_no_vacias_censo))  

# empty columns
print('Columnas vacias en el censo, :', list(set(censo_tapestry.drop(columns=['CVEGEO', 'tapestry', 'est', 'mun']).columns.to_list()) - set(col_no_vacias_censo)))

In [ ]:
from correct3 import adjustdf, find_next_ing

# Studying the cases that are in the census tapestry and not in the ENIGH tapestry
faltantes = list(set(col_no_vacias_censo)-set(col_no_vacias_entrenamiento))
faltantes

In [103]:
# use if function has error
enigh['ing_cor_t_segment'] = enigh['ing_cor_segment'].copy()

In [ ]:
# Correcting the enigh base so that it has all sectors
eni, censo_p = adjustdf(enigh, censo_tapestry.copy(), faltantes)
eni.shape, censo_p.shape

In [ ]:
# Checking that they have the same elements of the tapestry
list(set(eni['tapestry'].unique())-set(censo_p.drop(columns=['CVEGEO', 'tapestry', 'est', 'mun']).columns))

### 3.4 Obtaining the income and expense values

Expenditure and income columns from which we will obtain values

In [106]:
var = ['ing_cor', 'ingtrab',
       'trabajo', 'sueldos', 'horas_extr', 'comisiones', 'aguinaldo',
       'indemtrab', 'otra_rem', 'remu_espec', 'negocio', 'noagrop',
       'industria', 'comercio', 'servicios', 'agrope', 'agricolas',
       'pecuarios', 'reproducc', 'pesca', 'otros_trab', 'rentas', 'utilidad',
       'arrenda', 'transfer', 'jubilacion', 'becas', 'donativos',
       'remesas', 'bene_gob', 'transf_hog', 'trans_inst', 'estim_alqu',
       'otros_ing', 'gasto_mon', 'alimentos', 'ali_dentro', 'cereales',
       'carnes', 'pescado', 'leche', 'huevo', 'aceites', 'tuberculo',
       'verduras', 'frutas', 'azucar', 'cafe', 'especias', 'otros_alim',
       'bebidas', 'ali_fuera', 'tabaco', 'vesti_calz', 'vestido', 'calzado',
       'vivienda', 'alquiler', 'pred_cons', 'agua', 'energia', 'limpieza',
       'cuidados', 'utensilios', 'enseres', 'salud', 'atenc_ambu', 'hospital',
       'medicinas', 'transporte', 'publico', 'foraneo', 'adqui_vehi',
       'mantenim', 'refaccion', 'combus', 'comunica', 'educa_espa',
       'educacion', 'esparci', 'paq_turist', 'personales', 'cuida_pers',
       'acces_pers', 'otros_gas', 'transf_gas', 'percep_tot', 'retiro_inv',
       'prestamos', 'otras_perc', 'ero_nm_viv', 'ero_nm_hog', 'erogac_tot',
       'cuota_viv', 'mater_serv', 'material', 'servicio', 'deposito',
       'prest_terc', 'pago_tarje', 'deudas', 'balance', 'otras_erog', 'smg']

tapestry = list(eni['tapestry'].unique())

In [ ]:
eni[eni[var].sum(axis=1)==0]

In [ ]:
eni.loc[eni['tapestry']=='1_3_0', var] = eni.loc[(eni['tapestry']=='1_2_0') | (eni['tapestry']=='1_3_1'), var].mean().values

eni.loc[eni['tapestry']=='2_3_0', var] = eni.loc[(eni['tapestry']=='2_2_0') | (eni['tapestry']=='2_3_1'), var].mean().values

eni.loc[eni['tapestry']=='0_3_0', var] = eni.loc[(eni['tapestry']=='0_2_0') | (eni['tapestry']=='0_3_1'), var].mean().values

eni[eni[var].sum(axis=1)==0].shape

Data frame with the quantiles for each income segment by tapestry and variable

In [111]:
# Creating the dataframe with the slices we will use
var_ext =  [ i+str(j) for i in var for j in tapestry]
df = pd.DataFrame(np.zeros((7, len(var_ext))), columns=var_ext)

cuantiles = np.linspace(0,1, 7)

for i in var:
  for j in tapestry:
    # Saving the quartiles for each of the variables
    df.loc[:, i+str(j)] = list(np.quantile(eni.loc[eni['tapestry']==j, i], cuantiles))

Pasting the data from both bases

In [ ]:
# With merge we get errors, so we sort the bases and then merge them
(censo_p[['CVEGEO']].sort_values(by=['CVEGEO']) == censo[['CVEGEO']].sort_values(by=['CVEGEO'])).sum(), censo.shape[0]

In [113]:
censo_p = censo_p.sort_values(by=['CVEGEO']).reset_index(drop=True)
censo = censo.sort_values(by=['CVEGEO']).reset_index(drop=True)

censo_p[['clase_0', 'clase_1', 'clase_2', 'clase_3', 'clase_4', 'clase_5', 'ing_cor_t_segment']] = censo[['clase_0', 'clase_1', 'clase_2', 'clase_3', 'clase_4', 'clase_5', 'ing_cor_t_segment']].copy()
censo_p.shape

(15843, 73)

Creating the spend base

In [ ]:
start = time.time()

df3 = pd.DataFrame(np.zeros((censo_p.shape[0], len(var))), columns=var)
df3['CVEGEO'] = censo_p['CVEGEO'].copy().values
df3['tapestry'] = censo_p['tapestry'].copy().values
df3['ing_cor_t_segment'] = censo_p['ing_cor_t_segment'].copy().values

for i in tapestry:
# Select the predominant class 
# Segmenting by the income cutoff
  temp = censo_p[censo_p['tapestry'] == i].copy()
  for j in range(6):
    # Iterating over the possible classes
    for k in var:
        df3.loc[df3['tapestry'] == i, k] = df3.loc[df3['tapestry'] == i, k].copy().values + np.random.uniform(df.loc[j, k+i], df.loc[j+1, k+i], temp.shape[0])*temp['clase_'+str(j)].copy().values
  print(i)

end = time.time()
print('Running time: {:.2f} minutes'.format((end-start)/60))

In [ ]:
df

In [ ]:
df3

In [117]:
# guarde el de la media y la mediana
filename = '/content/gdrive/MyDrive/pamgea3_0/pangea3_2/ingasto/estado'+str(estado)+'.csv' # Siguiendo '3.2.2 Tapestry procedimiento original'
df3.to_csv(filename, index=False)

### Graphing the results

In [118]:
estado=1
censo2 = pd.read_csv('/content/gdrive/MyDrive/pamgea3_0/pangea3_2/ingasto/estado'+str(estado)+'.csv')

In [ ]:
acumulado = censo2['tapestry'].value_counts(normalize=True)
acumulado2 = enigh['tapestry'].value_counts(normalize=True)

censo_ac = pd.DataFrame(acumulado)
enigh_ac = pd.DataFrame(acumulado2)
censo_ac.columns = ['Censo']
enigh_ac.columns = ['Enigh']
enigh_ac.merge(censo_ac, left_index=True, right_index=True, how='outer').sort_index().plot.bar(figsize=[20,5])

### Comparing Income and Expenditure

In [ ]:
from numpy.core.fromnumeric import mean

# ENIGH variables
enigh['ing_gasto'] = enigh['ing_cor'] - enigh['gasto_mon']
enigh['ing_gasto'] = np.where(enigh['ing_gasto'] < 0, 1, 0)
print('Cases where spending is greater than income', enigh['ing_gasto'].sum())
print('Percentage of data with more expenses than income', enigh['ing_gasto'].sum()/enigh.shape[0])
print('How much more is the expense than the average income in those cases?', (enigh.loc[enigh['ing_gasto']==1, 'ing_cor'] - enigh.loc[enigh['ing_gasto']==1, 'gasto_mon']).abs().mean())
print('Average in cases where income is greater than expenditure', (enigh.loc[enigh['ing_gasto']==0, 'ing_cor'] - enigh.loc[enigh['ing_gasto']==0, 'gasto_mon']).abs().mean())
print('General description')
(enigh['ing_cor'] - enigh['gasto_mon']).describe()

In [ ]:
### Census variables
censo2['ing_gasto'] = censo2['ing_cor'] - censo2['gasto_mon']
censo2['ing_gasto'] = np.where(censo2['ing_gasto'] < 0, 1, 0)
print('Cases where spending is greater than income', censo2['ing_gasto'].sum())
print('Percentage of data with more expenses than income', censo2['ing_gasto'].sum()/censo2.shape[0])
print('How much more is the expense than the average income in those cases?', (censo2.loc[censo2['ing_gasto']==1, 'ing_cor'] - censo2.loc[censo2['ing_gasto']==1, 'gasto_mon']).abs().mean())
print('Average in cases where income is greater than expenditure', (censo2.loc[censo2['ing_gasto']==0, 'ing_cor'] - censo2.loc[censo2['ing_gasto']==0, 'gasto_mon']).abs().mean())
print('General description')
(censo2['ing_cor'] - censo2['gasto_mon']).describe()

In [ ]:
x0 = list(range(censo2.shape[0]))

plt.figure(figsize=(30, 10))
plt.scatter(x0, censo2['ing_cor'], color = censo2['ing_gasto'].replace({0:'pink', 1:'crimson'}), alpha=0.3, label="Ingreso corriente")
plt.scatter(x0, censo2['gasto_mon'], color = censo2['ing_gasto'].replace({0:'lightskyblue', 1:'royalblue'}), alpha=0.3, label="Gasto monetario")
plt.ylabel("Pesos trimestrales", size=20)
plt.legend(loc='upper right')
plt.show()

In [ ]:
a = enigh.loc[enigh['ing_gasto']==1, 'ing_cor_segment'].value_counts()

plt.figure(figsize=(8,5))
plt.bar(a.index, a.values, linewidth=25, color = 'yellowgreen', alpha = 0.8)
plt.xticks(a.index)
plt.xlabel('Segmento de ingreso')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
a = censo2.loc[censo2['ing_gasto']==1, 'ing_cor_t_segment'].value_counts()

plt.figure(figsize=(8,5))
plt.bar(a.index, a.values, linewidth=25, color = 'yellowgreen', alpha = 0.8)
plt.xticks(a.index)
plt.xlabel('Segmento de ingreso')
plt.ylabel('Frecuencia')
plt.show()

### Total expenditure and income as the sum of their components

In [ ]:
difingcor = enigh['ing_cor']-enigh['ingtrab']-enigh['rentas']-enigh['transfer']-enigh['estim_alqu']-enigh['otros_ing']
print('Enigh income difference:', difingcor.mean()/3, difingcor.max()/3, difingcor.std()/3)

difingcor = censo2['ing_cor']-censo2['ingtrab']-censo2['rentas']-censo2['transfer']-censo2['estim_alqu']-censo2['otros_ing']
print('Income difference in Pangea:', difingcor.mean()/3, difingcor.max()/3, difingcor.std()/3)

In [ ]:
difgasto = enigh['gasto_mon']-enigh['alimentos']-enigh['vesti_calz']-enigh['vivienda']-enigh['limpieza']-enigh['salud']-enigh['transporte']-enigh['educa_espa']-enigh['personales']-enigh['transf_gas']
print('Enigh income difference:', difgasto.mean()/3,difgasto.std()/3,difgasto.max()/3)

difgasto = censo2['gasto_mon']-censo2['alimentos']-censo2['vesti_calz']-censo2['vivienda']-censo2['limpieza']-censo2['salud']-censo2['transporte']-censo2['educa_espa']-censo2['personales']-censo2['transf_gas']
print('Pangea Income Difference:', difgasto.mean()/3,difgasto.std()/3,difgasto.max()/3)

In [ ]:
difalim = enigh['ali_dentro']-enigh['cereales']-enigh['carnes']-enigh['pescado']-enigh['leche']-enigh['huevo']-enigh['aceites']-enigh['tuberculo']-enigh['verduras']-enigh['frutas']-enigh['azucar']-enigh['cafe']-enigh['especias']-enigh['otros_alim']-enigh['bebidas']
print('Difference of food inside Enigh:', difalim.mean()/3, difalim.std()/3, difalim.max()/3)

difalim = censo2['ali_dentro']-censo2['cereales']-censo2['carnes']-censo2['pescado']-censo2['leche']-censo2['huevo']-censo2['aceites']-censo2['tuberculo']-censo2['verduras']-censo2['frutas']-censo2['azucar']-censo2['cafe']-censo2['especias']-censo2['otros_alim']-censo2['bebidas']
print('Difference of food inside real:', difalim.mean()/3, difalim.std()/3, difalim.max()/3)

## Comparing current income segments

In [ ]:
for i in range(6):
  plt.hist([enigh.loc[enigh['ing_cor_segment']==i, 'ing_cor'], censo2.loc[censo2['tapestry'].str[0]==str(i), 'ing_cor']], label=['Enigh', 'Censo'], bins=20, density=True)
  plt.title('Ingreso corriente del segmento '+str(i))
  plt.legend(loc='upper right')
  plt.show()

### Calculating the metrics (all variables) Pondera

In [ ]:
var = ['ing_cor', 'ingtrab', 'rentas', 'transfer', 'estim_alqu', 'otros_ing',
       'gasto_mon', 'alimentos', 'vesti_calz', 'vivienda', 'limpieza', 'salud', 'transporte', 'educa_espa', 'personales', 'transf_gas',
       'ali_dentro', 'pago_tarje']

nclasses = 6

for j in var:
  print(j)
  xt = enigh[j].values
  breaks = jenkspy.jenks_breaks(xt, nclasses)
  breaks = np.array(breaks)
  
  # Segmentation
  enigh[j+'_segment'] = enigh[j].apply(find_segment,args=(breaks,))
  censo2[j+'_segment'] = censo2[j].apply(find_segment,args=(breaks,))

  # data base segmentation variables
  censo_pred = (censo2[j+'_segment'].value_counts(normalize=True)*100).sort_index()
  enigh_true = (enigh[j+'_segment'].value_counts(normalize=True)*100).sort_index()
  censo_pred.columns = ['Censo']
  enigh_true.columns = ['Enigh']
  censo_pred = pd.DataFrame(censo_pred).reset_index()
  enigh_true = pd.DataFrame(enigh_true).reset_index()

  temp = pd.DataFrame({'index':np.arange(6)})

  temp = temp.merge(censo_pred, how='left', on='index')
  temp = temp.merge(enigh_true, how='left', on='index')

  temp = temp.fillna(0)


  met = []
  for k in range(6):
      met.append((temp.loc[k, j+'_segment_y']-temp.loc[k, j+'_segment_x'])/(temp.loc[k, j+'_segment_y']+temp.loc[k, j+'_segment_x']))
      print('Segment metric', k,':', met[k])

  val=0
  for k in range(6):
      val = val + np.abs(met[k])
  print('Generarl metric:', val/6)

  val=0
  for k in range(6):
      val = val + np.abs(met[k])*(temp.loc[k, j+'_segment_y']/temp[j+'_segment_y'].sum())
  print('Balanced metric:', val, '\n')

### Graphs of current income and monetary expenditure

In [132]:
muns = ['001']

In [ ]:
estado_ = '0'+str(estado) if estado<10 else str(estado)
shp = gpd.read_file(f'/content/gdrive/MyDrive/pamgea3_0/shape_estados/{estado_}m.shp')
shp.to_crs('EPSG:4326',inplace=True)

if len(muns)>0:
    shp = shp[shp['CVE_MUN'].isin(muns)].reset_index(drop=True)
    
shp = shp.merge(censo2[['CVEGEO', 'ing_cor', 'gasto_mon']], left_on='CVEGEO',right_on='CVEGEO')
base = shp.plot(column='ing_cor',figsize=[15,15],legend=True)

In [ ]:
shp.plot(column='gasto_mon',figsize=[15,15],legend=True)